[View in Colaboratory](https://colab.research.google.com/github/juzkev/HDB-Resale-Prices/blob/master/HDB_Resale_Prices.ipynb)

# Predicting HDB Resale prices

In [102]:
!pip install requests

import requests
import pandas as pd

df = pd.DataFrame()

rids = ['1b702208-44bf-4829-b620-4615ee19b57c', '83b2fc37-ce8c-4df4-968b-370fd818138b', '8c00bf08-9124-479e-aeca-7cc411d884c4', 'adbbddd3-30e2-445f-a123-29bee150a6fe']
url = 'https://data.gov.sg/api/action/datastore_search'

for rid in rids[0:1]:
  print('Downloading from rid {}'.format(rid))
  params = params={
      'resource_id': rid, 
       'limit': 500000,
  }

  with requests.get(url, params=params) as r:
    df = pd.concat([df,pd.DataFrame(r.json()['result']['records'])])  
    type_dict = ({field['id']: field['type'] for field in r.json()['result']['fields']})

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple


In [0]:
# Setting column dtype
for col in ('floor_area_sqm', 'remaining_lease', 'resale_price'):
    df[col] = pd.to_numeric(df[col])
df['lease_commence_date'] = pd.to_datetime(df['lease_commence_date']).dt.floor('d')
df['month'] = pd.to_datetime(df['month']).dt.floor('d')

In [117]:
df.infer_objects().info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59809 entries, 0 to 59808
Data columns (total 12 columns):
_id                    59809 non-null int64
block                  59809 non-null object
flat_model             59809 non-null object
flat_type              59809 non-null object
floor_area_sqm         59809 non-null float64
lease_commence_date    59809 non-null datetime64[ns]
month                  59809 non-null datetime64[ns]
remaining_lease        59809 non-null int64
resale_price           59809 non-null float64
storey_range           59809 non-null object
street_name            59809 non-null object
town                   59809 non-null object
dtypes: datetime64[ns](2), float64(2), int64(2), object(6)
memory usage: 5.5+ MB


,_id,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
count,59809.000000,59809,59809,59809,59809.000000,59809,59809,59809.000000,5.980900e+04,59809,59809,59809
unique,NaN,2186,20,7,NaN,50,38,NaN,NaN,17,524,26
top,NaN,2,Model A,4 ROOM,NaN,1985-01-01 00:00:00,2017-11-01 00:00:00,NaN,NaN,04 TO 06,YISHUN RING RD,JURONG WEST
freq,NaN,241,18042,24628,NaN,4175,1975,NaN,NaN,14195,984,4668
first,NaN,NaN,NaN,NaN,NaN,1966-01-01 00:00:00,2015-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2015-01-01 00:00:00,2018-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,29905.000000,NaN,NaN,NaN,97.319194,NaN,NaN,74.017890,4.394339e+05,NaN,NaN,NaN
std,17265.515462,NaN,NaN,NaN,24.166368,NaN,NaN,11.274598,1.415803e+05,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,31.000000,NaN,NaN,47.000000,9.050000e+04,NaN,NaN,NaN
25%,14953.000000,NaN,NaN,NaN,75.000000,NaN,NaN,66.000000,3.400000e+05,NaN,NaN,NaN


In [113]:
df.sample(10)

,_id,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
56448,56449,83,Standard,3 ROOM,64.0,1968-01-01,2017-12-01,49,250000.0,01 TO 03,MACPHERSON LANE,GEYLANG
37913,37914,81,Standard,3 ROOM,60.0,1967-01-01,2017-01-01,49,307000.0,04 TO 06,C'WEALTH CL,QUEENSTOWN
54581,54581,67,Standard,3 ROOM,59.0,1969-01-01,2017-11-01,50,220000.0,01 TO 03,CIRCUIT RD,GEYLANG
6200,6199,94,Improved,4 ROOM,91.0,1974-01-01,2015-05-01,57,425000.0,16 TO 18,WHAMPOA DR,KALLANG/WHAMPOA
41607,41607,34,Standard,3 ROOM,59.0,1971-01-01,2017-04-01,52,372000.0,04 TO 06,JLN BT HO SWEE,BUKIT MERAH
31080,31081,303,Improved,5 ROOM,121.0,1985-01-01,2016-09-01,67,755000.0,04 TO 06,SHUNFU RD,BISHAN
24142,24143,206,Model A,4 ROOM,103.0,1989-01-01,2016-05-01,71,345000.0,04 TO 06,PETIR RD,BUKIT PANJANG
28613,28614,540,Improved,5 ROOM,122.0,1992-01-01,2016-07-01,75,535000.0,04 TO 06,SERANGOON NTH AVE 4,SERANGOON
55691,55692,125,Improved,3 ROOM,60.0,1985-01-01,2017-11-01,66,245000.0,04 TO 06,MARSILING RISE,WOODLANDS
47416,47417,51,Standard,5 ROOM,117.0,1977-01-01,2017-07-01,58,600000.0,13 TO 15,SIMS DR,GEYLANG
